## Importación de librerías 

En esta sección se importan las principales librerías utilizadas en el preprocesamiento del texto.
- **spaCy**: biblioteca de procesamiento del lenguaje natural (NLP) que permite realizar tareas como tokenización, eliminación de stopwords o lematización.
    - Se incluye además la configuración personalizada del tokenizador de spaCy, que permite ajustar cómo se separan los tokens (por ejemplo, evitar divisiones innecesarias en palabras con guiones, puntos, etc.).
- **pandas**: biblioteca para la manipulación de datos estructurados (tablas) de tipo DataFrame.
- **collections.Counter**: conteo eficiente de elementos en colecciones.
- **sklearn.preprocessing.normalize**: normalización de vectores o matrices.
- **sklearn.metrics.pairwise.cosine_similarity**: cálculo de similitud coseno entre vectores.


In [8]:
# Librería que contiene herramientas de preprocesamiento
import spacy

# Herramientas internas de spaCy para modificar el tokenizador por defecto
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

# Librería para carga de datos
import pandas as pd

# Librería para contar frecuencias
from collections import Counter

# Librerías para normalizar y aplicar similitud del coseno
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity


## Definición de funciones y configuración de preprocesamiento

Este bloque incluye todo lo necesario para preparar el texto antes de su análisis. Se utiliza el modelo `en_core_web_lg` de spaCy y se personaliza el tokenizador para evitar divisiones en guiones.

### Componentes definidos:

- **Carga y personalización del modelo spaCy**: se modifica el tokenizador para no dividir palabras por guiones.
- **`tokenizar_texto_spacy(texto)`**: convierte una cadena de texto a minúsculas y la transforma en una lista de tokens.
- **`eliminar_stopwords_spacy(tokens)`**: elimina tokens irrelevantes, incluyendo stopwords, palabras no alfabéticas y una lista personalizada de palabras excluidas.
- **`lematizar_texto_spacy(tokens)`**: transforma cada token en su lema o raíz.
- **`aplicar_mapeo_df(serie_de_listas, mapeo)`**: aplica un diccionario de equivalencias léxicas para unificar términos similares.
- **`palabras_extra`**: lista manual de palabras irrelevantes específicas del contexto del corpus.
- **`mapeo`**: diccionario de normalización construido a partir de un archivo `.csv` externo.


In [9]:
# Cargar el modelo grande de inglés (incluye vectores de palabras)
nlp = spacy.load("en_core_web_lg")

# Eliminar los guiones como separadores internos para evitar fragmentación de tokens como "ai-based" en "ai" "based"
infixes = [x for x in nlp.Defaults.infixes if "-" not in x]
infix_re = compile_infix_regex(infixes)

# Reconfigurar el tokenizador con las nuevas reglas de infijos
nlp.tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=nlp.tokenizer.prefix_search,
    suffix_search=nlp.tokenizer.suffix_search,
    infix_finditer=infix_re.finditer,
    token_match=nlp.tokenizer.token_match
)



# Función de tokenización básica con spaCy en la que también se pasa el texto en minúsculas
# La separación en tokens se realiza según las reglas del modelo de lenguaje cargado,
# teniendo en cuenta espacios, puntuación, signos de puntuación interna (como guiones, apóstrofes, etc.)
# y excepciones predefinidas. En este caso, se ha modificado el tokenizer para que no divida por guiones.
def tokenizar_texto_spacy(texto):
    doc = nlp(texto.lower())
    return list(doc)




# Lista personalizada de palabras a ignorar (además de las stopwords de spaCy)
palabras_extra = [
    "co", "digital", "alexa", "anytime", "cortana", "arduino", "bitcoin",
    "deepl", "digitally", "digitise", "dropbox", "duckduckgo", "e", "etc",
    "eurostat", "example", "facebook", "google", "miro", "t", "twitter",
    "tpm", "youtube", "vs", "whatsapp", "wikipedia", "x", "en", "se", "lego",
    "non", "python", "ros", "scratch", "siri", "-", "non-digital", "one-time",
    "bas", "digital-based", "digitise", "digitised", "digitization", "s", "whilst"
]

# Asegurar que "3d" y "3-d" no se consideren stopwords, ya que el eliminador de stopwrods 
# elimina todas con números
nlp.vocab["3d"].is_alpha = True
nlp.vocab["3-d"].is_alpha = True
nlp.vocab["2d"].is_alpha = True
nlp.vocab["2-d"].is_alpha = True

# Función para eliminar stopwords y tokens no alfabéticos.
# Ignora las palabras_extra
def eliminar_stopwords_spacy(tokens):
    return [
        token for token in tokens
        if not token.is_stop
        and token.is_alpha
        and token.text.lower() not in palabras_extra
    ]
    
    
    
    
# Función de lematización del modelos cargado
def lematizar_texto_spacy(tokens):
    return [token.lemma_ for token in tokens]


# Función de mapeo de palabras en función de un diccionario de mapeo para normalización de términos
def aplicar_mapeo(serie_de_listas, mapeo):
    return serie_de_listas.apply(lambda tokens: [mapeo.get(token, token) for token in tokens])



#carga del archivo que contiene los mapeos correspondientes y conversión a tipo DataFrame para ser tratado como tabla
ruta_mapeo = "../utils/mapping.csv"
df_mapeo = pd.read_csv(ruta_mapeo)

# Construcción del diccionario: original -> equivalente
mapeo = dict(zip(df_mapeo["original"], df_mapeo["equivalente"]))

## Función `comparar_texto_con_frecuencias`

Compara un texto con grupos de una tabla de frecuencias y calcula la similitud basada en palabras comunes.

**Parámetros:**

- `texto`: lista de palabras del texto preprocesado.
- `df_pivot`: DataFrame con palabras como filas y grupos como columnas, con frecuencias.
- `top_n`: número de palabras clave a mostrar por grupo (por defecto 5).

**Funcionamiento resumido:**

1. Obtiene el vocabulario del DataFrame.
2. Crea un vector con las frecuencias de las palabras del texto en el vocabulario.
3. Combina este vector con los vectores de frecuencias de los grupos.
4. Normaliza los vectores para calcular similitud coseno.
5. Calcula la similitud entre el texto y cada grupo.
6. Identifica las palabras que más contribuyen a la similitud para cada grupo.
7. Devuelve un DataFrame con grupo, similitud y palabras clave ordenado por similitud.

**Devuelve:**

Un DataFrame con columnas: `'Grupo'`, `'Similitud'` y `'Palabras_clave'`.

## Función `aplicar_mapeo_a_texto(lista_tokens, mapeo)`

Reemplaza cada token en una lista de estos según un diccionario de mapeo léxico para unificar términos similares.


In [10]:
def comparar_texto_con_frecuencias(texto, df_pivot, top_n=5):

    # 1. Vocabulario (palabras que aparecen en df_pivot)
    vocabulario = df_pivot.index.tolist()

    # 2. Vector del texto: contar solo palabras del vocabulario
    conteo_texto = Counter(p for p in texto if p in vocabulario)
    vector_texto = pd.Series([conteo_texto.get(p, 0) for p in vocabulario], index=vocabulario).to_frame(name="Texto")

    # 3. Concatenar vectores del texto y de grupos
    matriz_completa = pd.concat([df_pivot, vector_texto], axis=1).fillna(0)

    # 4. Normalizar vectores para cálculo coseno
    matriz_normalizada = normalize(matriz_completa.T)

    # 5. Calcular similitud coseno entre texto (última fila) y cada grupo (todas menos última)
    similitudes = cosine_similarity([matriz_normalizada[-1]], matriz_normalizada[:-1])[0]

    # 6. Preparar resultados básicos
    nombres_grupo = df_pivot.columns.tolist()
    resultados = pd.DataFrame({
        'Grupo': [g if isinstance(g, str) else ' - '.join(map(str, g)) for g in nombres_grupo],
        'Similitud': similitudes
    })

    # 7. Calcular contribuciones por palabra a la similitud de cada grupo
    #    La contribución para cada palabra = frecuencia_texto * frecuencia_grupo
    #    Ordenamos y extraemos las top_n palabras para cada grupo
    palabras_clave_por_grupo = []
    for grupo in nombres_grupo:
        contribuciones = []
        for palabra in vocabulario:
            f_texto = vector_texto.at[palabra, "Texto"]
            f_grupo = df_pivot.at[palabra, grupo]
            if f_texto > 0 and f_grupo > 0:
                contribuciones.append((palabra, f_texto * f_grupo))

        # Ordenar por contribución descendente
        contribuciones.sort(key=lambda x: x[1], reverse=True)
        top_palabras = [p for p, _ in contribuciones[:top_n]]
        palabras_clave_por_grupo.append(", ".join(top_palabras))

    resultados["Palabras_clave"] = palabras_clave_por_grupo

    # 8. Ordenar resultados por similitud descendente
    resultados = resultados.sort_values(by="Similitud", ascending=False).reset_index(drop=True)

    return resultados

def aplicar_mapeo_a_texto(lista_tokens, mapeo):
    return [mapeo.get(token, token) for token in lista_tokens]

## Bloque para modificar el texto a analizar

En este bloque el usuario debe introducir el texto que desea analizar. El texto pasará por varias etapas de preprocesamiento: tokenización, eliminación de stopwords, lematización y aplicación del mapeo léxico definido.


In [18]:
# TEXTO A COMPARAR
texto = "Networks, communications and Internet Telecommunications networks. Concepts. Transmission media. Circuit and packet switching. Routing protocols. Access infrastructures. Network interconnection. Quality of service. The Internet and basic services. Cabling systems and network interconnection equipment. The ISO Open Systems Interconnection (OSI) reference model: architecture, layers, interfaces, protocols, addressing and routing. Access technologies: fibre (GPON, FTTH), mobile (LTE), wireless. Transport networks: JDSxWDM, MPLS. Aggregation networks: ATM, Carrier Ethernet-VPLS (H-VPLS). Wireless networks: the IEEE 802.11 standard. Functional and technical characteristics. Spectrum expansion systems. Access systems. Authentication. Modes of operation. Bluetooth. Security, regulatory standards. IP networks: network architecture, routing and quality of service. IPv4 - IPv6 transition and coexistence. Specific IPv6 functionalities. Next-generation networks and converged services (NGN/IMS). VoIP, ToIP and unified communications. Digital transformation and Industry 4.0: smart cities. Internet of Things (IoT). Local area networks. Architecture. Types. Transmission media. Access methods. Interconnection devices. Device management. LAN network administration. User management in local networks. Traffic monitoring and control. SNMP management. Configuration and management of virtual networks (VLAN). Wide area networks. Intranet and Extranet network architecture. Concept, structure and characteristics. Their implementation in organisations. Layer model: application servers, data servers, server farms. Public data transmission networks. The SARA network. The sTESTA network. Network planning and management. Cable telecommunications (CATV). Cable network structure. Market operators. Network services. Email. Messaging services. Directory services. Mobile communications. Generations of mobile phone technologies. Mobile applications. Features, technologies, distribution and trends. Network security. Types of attacks and tools for their prevention: firewalls, access and intrusion control, cryptographic techniques, etc. Specific measures for mobile communications. Application-level security. Types of attacks and protection of web services, databases and user interfaces. Cybersecurity. The national cybersecurity strategy. Business continuity management. Business Continuity and Contingency Plans. Telecommunications regulatory standards. The National Commission for Markets and Competition (CNMC): organisation, functions and competence in the field of telecommunications. Video conferencing systems. Group work tools. Dimensioning and quality of service in communications and conditioning of rooms and equipment. Video streaming. Remote access to corporate systems: identity management, single sign-on and teleworking. System and data centre virtualisation. Workstation virtualisation. Windows terminal and Linux server models. Collaborative work tools and social networks. The State Administration's Digital Communication Guide."
texto1 = tokenizar_texto_spacy(texto)
texto2 = eliminar_stopwords_spacy(texto1)
texto3 = lematizar_texto_spacy(texto2)
texto4 = aplicar_mapeo_a_texto(texto3, mapeo)

## Comparación de texto seleccionado con texto de ESCO

In [19]:
ruta_frecuencias_esco = "../results/esco/frecuencias_esco.csv"
frecuencias_esco = pd.read_csv(ruta_frecuencias_esco, index_col=0)

resultados_esco = comparar_texto_con_frecuencias(texto4, frecuencias_esco, top_n=5)

pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_colwidth', None)  # Muestra todo el contenido de las celdas, sin truncar

print(resultados_esco)

                                                                                                            Grupo  \
0                                                                                             computer technology   
1                                                                                       design information system   
2                                                                  database and network design and administration   
3                                                                                             maintain ICT system   
4                                                                                          protecting ict devices   
5                                                                                              ICT infrastructure   
6                                                                                 apply digital security measures   
7                                                               